In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from google.colab import userdata
from transformers import DataCollatorForSeq2Seq

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cfilt/iitb-english-hindi", token=userdata.get("huggingface_token"))
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, token=userdata.get("huggingface_token"))

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
max_input_len = 128
max_output_len = 128

input_lang = "en"
target_lang = "hi"

In [ ]:
def preprocess_function(examples):
  en = [ex[input_lang] for ex in examples["translation"]]
  fr = [ex[target_lang] for ex in examples["translation"]]
  model_inputs = tokenizer(en, padding="max_length", truncation=True, return_tensors="pt")

  #setup tokenizer for the tartgets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(fr, padding="max_length", truncation=True, return_tensors="pt")

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
preprocess_function(dataset["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': tensor([[ 3872,    85,  2501,  ..., 61949, 61949, 61949],
        [32643, 28541, 36253,  ..., 61949, 61949, 61949]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[   63,  2025,    18,  ..., 61949, 61949, 61949],
        [26618, 16155,   346,  ..., 61949, 61949, 61949]])}

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["translation"])

In [ ]:
from torch.utils.data import TensorDataset
tokenized_dataset = TensorDataset(tokenized_dataset["input_ids"], tokenized_dataset["labels"], return_tensors="pt")

KeyError: 'input_ids'

In [ ]:
tokenized_dataset["train"][:2]

{'input_ids': [[3872,
   85,
   2501,
   132,
   15441,
   36398,
   0,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949,
   61949

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
learning_rate = 1e-3
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt", pad_to_multiple_of=128)

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(small_test_dataset, batch_size=8)

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)


In [ ]:
from transformers import get_scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader)*num_train_epochs
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./translation_save",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Adjust based on your GPU memory
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=small_train_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss


Step,Training Loss


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(len(train_dataloader)))
model.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
for epoch in range(num_train_epochs):
  for batch in train_dataloader:
    print(type(batch))
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

<class 'dict'>


AttributeError: 'list' object has no attribute 'to'

In [ ]:
trainer.save_model("My_model")

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM
model2 = TFAutoModelForSeq2SeqLM.from_pretrained("My_model")

In [ ]:
from transformers import pipeline
generator = pipeline("translation", model=model2, tokenizer=tokenizer)

In [ ]:
print(generator(input("Please input a sentence")))